In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/indonesian-marketplace-product-reviews/reviews.csv')

In [ ]:
df

## Buat semua reviews menjadi lowertext

In [ ]:
# df["reviews"] = df["reviews"].apply(str.lower)

In [ ]:
def to_lower(text):
    return text.lower()

df.reviews = df.reviews.apply(to_lower)
df.reviews[0]

## Menghapus special text

In [ ]:
def is_special(text):
    rem = ''
    for i in text:
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
    return rem

df.reviews = df.reviews.apply(is_special)
df.reviews[0]

## Hapus stopwords

In [ ]:
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

# def rem_stopwords(text):
#     stop_words = set(stopwords.words('indonesia'))
#     words = word_tokenize(text)
#     return [w for w in words if w not in stop_words]

# df.reviews = df.reviews.apply(rem_stopwords)
# df.reviews[0]

In [ ]:
!pip install Sastrawi

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
stop_factory = StopWordRemoverFactory()
more_stopword = ['dengan', 'ia','bahwa','oleh', 'nya', 'sih']
data = stop_factory.get_stop_words()+more_stopword
stopword = stop_factory.create_stop_word_remover()
print(data)

In [ ]:
# from nltk.tokenize import word_tokenize

# def rem_stopwords(text):
#     words = word_tokenize(text)
#     return [w for w in words if w not in data]

# df.reviews = df.reviews.apply(rem_stopwords)
# df.reviews[0]

In [ ]:
df.reviews

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_list(word_list):
    return [stemmer.stem(word) for word in word_list]

# Melakukan stemming pada setiap daftar kata dalam data
stemmed_data = [stem_list(words) for words in df.reviews]

In [ ]:
stemmed_data

In [ ]:
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt

# # membuat visualisasi WordCloud
# join_review = ''.join(''.join(l) for l in df.reviews)

# w_cloud = WordCloud(background_color="white",
#                     width=700, height=300, max_font_size=120,
#                     max_words=500, contour_color='blue',
#                     contour_width=2).generate(str(join_review))

# # plot wordcloud
# plt.figure(figsize=(12,10))
# plt.axis("off")
# plt.tight_layout()
# plt.imshow(w_cloud, interpolation="bilinear")
# plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Meratakan nested list menjadi list of string
flattened_reviews = [" ".join(review) for review in df.reviews]

# Membuat DataFrame dari flattened list
df_flattened = pd.DataFrame({'reviews': flattened_reviews})

# Inisialisasi CountVectorizer
cv = CountVectorizer()

# Transformasi data dan ubah menjadi array
X = cv.fit_transform(df_flattened['reviews']).toarray()

# Menampilkan hasil
print(X)

In [ ]:
df_flattened.head()

## Creating model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score
import pickle

In [ ]:
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(df['reviews']).toarray()
# y = df['sentiment']

In [ ]:
X = np.array(df.iloc[:,0].values)
y = np.array(df.label.values)
cv = CountVectorizer(max_features = 1000)
X = cv.fit_transform(df_flattened.reviews).toarray()
print("X.shape = ",X.shape)
print("y.shape = ",y.shape)

## train test split

In [ ]:
trainx,testx,trainy,testy = train_test_split(X,y,test_size=0.2,random_state=42)
print("Train shapes : X = {}, y = {}".format(trainx.shape,trainy.shape))
print("Test shapes : X = {}, y = {}".format(testx.shape,testy.shape))

## Membuat dan melakukan training pada model

In [ ]:
# gnb,mnb,bnb = GaussianNB(),MultinomialNB(alpha=1.0,fit_prior=True),BernoulliNB(alpha=1.0,fit_prior=True)
# gnb.fit(trainx,trainy)
# mnb.fit(trainx,trainy)
# bnb.fit(trainx,trainy)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import nltk
from nltk.classify import SklearnClassifier
from random import shuffle


In [ ]:
features = [(dict(enumerate(x)), y) for x, y in zip(X, y)]
shuffle(features)
train_count = len(features) * 3 // 4
train_data = features[:train_count]
test_data = features[train_count:]

In [ ]:
features

In [ ]:
classifiers = {
    "BernoulliNB": BernoulliNB(),
    "ComplementNB": ComplementNB(),
    "MultinomialNB": MultinomialNB(),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "MLPClassifier": MLPClassifier(max_iter=1000),
    "AdaBoostClassifier": AdaBoostClassifier(),
}

best_accuracy = 0
best_classifier = None
best_classifier_name = ""

for name, sklearn_classifier in classifiers.items():
    classifier = SklearnClassifier(sklearn_classifier)
    classifier.train(train_data)
    accuracy = nltk.classify.accuracy(classifier, test_data)
    print(f"{accuracy:.2%} - {name}")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_classifier = classifier
        best_classifier_name = name

print(f"\nModel terbaik adalah {best_classifier_name} dengan akurasi {best_accuracy:.2%}")

In [ ]:
classifier_instance = best_classifier._clf

In [ ]:
classifier_instance

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

# Menggabungkan kembali data pelatihan dan pengujian untuk melatih model terbaik dengan seluruh data
all_X = vectorizer.fit_transform(df_flattened['reviews']).toarray()
all_y = df['label']

# Melatih model terbaik dengan seluruh data
# best_classifier = MultinomialNB()
classifier_instance.fit(all_X, all_y)


In [ ]:
# Data baru untuk diuji
new_reviews = [
    "Pengiriman sangat cepat dan tepat waktu.",
    "Produk ini benar-benar buruk dan mengecewakan.",
    "Produknya sangat bagus dan kualitasnya pun sesuai harga.",
    "Harga terlalu mahal untuk kualitas yang diberikan."
]

# Preprocessing data baru
new_X = vectorizer.transform(new_reviews).toarray()

# Prediksi sentimen dengan model terbaik
new_predictions = classifier_instance.predict(new_X)

# Menampilkan hasil prediksi
for review, sentiment in zip(new_reviews, new_predictions):
    sentiment_label = "Positif" if sentiment == 1 else "Negatif"
    print(f"Review: '{review}' -> Sentimen: {sentiment_label}")


## Memilih model terbaik

In [ ]:
# ypg = gnb.predict(testx)
# ypm = mnb.predict(testx)
# ypb = bnb.predict(testx)

# print("Gaussian = ",accuracy_score(testy,ypg))
# print("Multinomial = ",accuracy_score(testy,ypm))
# print("Bernoulli = ",accuracy_score(testy,ypb))

## Uji coba model

In [ ]:
# pickle.dump(mnb,open('model.pkl','wb'))

In [ ]:
# senti_neg = "Produk ini sangat mengecewakan, kualitasnya jauh di bawah harapan"
# senti_pos = "Pengalaman belanja yang menyenangkan, saya pasti akan membeli dari toko ini lagi"

In [ ]:
# text = senti_pos

# bow,words = [],word_tokenize(text)
# for word in words:
#     bow.append(words.count(word))
# # np.array(bow).reshape(1,3000)
# # bow.shape
# word_dict = cv.vocabulary_
# pickle.dump(word_dict,open('bow.pkl','wb'))

In [ ]:
# word_dict

In [ ]:
# inp = []
# for i in word_dict:
#     inp.append(senti_pos.count(i[0]))
# y_pred = mnb.predict(np.array(inp).reshape(1,1000))
# print(y_pred)

In [ ]:
# inp = []
# for i in word_dict:
#     inp.append(senti_neg.count(i[0]))
# y_pred = mnb.predict(np.array(inp).reshape(1,1000))
# print(y_pred)